In [ ]:
from llmcam.ytlive import capture_youtube_live_frame
from llmcam.gpt4v import ask_gpt4v
import openai
from typing import Optional
import json

In [ ]:
def extract_live_info(link: Optional[str] = None):
    if link:
        image = capture_youtube_live_frame(link)
    else:
        image = capture_youtube_live_frame()
    
    return ask_gpt4v(image)

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "extract_live_info",
            "description": "Extarct information from a YouTube Live",
            "parameters": {
                "type": "object",
                "properties": {
                    "link": {
                        "anyOf": [
                            {
                                "type": "string",
                                "description": "YouTube Live URL"
                            },
                            {
                                "type": "null",
                                "description": "No URL provided, default used"
                            }
                        ]
                    }
                },
                "required": [],
                "additionalProperties": False,
            },
        }
    }
]

functions = {
    "extract_live_info": extract_live_info
}

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": "Hi, can you tell me the information in YouTube Live? Here is the link https://www.youtube.com/watch?v=LMZQ7eFhm58"}
]

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    functions=functions,
    function_call="auto"
)

messages.append({"role": response.choices[0].message.role, "content": response.choices[0].message.content})
messages.append({"role": "user", "content": "I don't have a specific URL. Can you still tell me the information in some readily available Live?"})

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
) # First response without link

# ChatGPT called a function so there is no response from assistant
messages.append({"role": "user", "content": "Thank you for that Live information. I have found this Live that seems interesting. Here is the link https://www.youtube.com/watch?v=dFBRpHHwQeg."})

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
) # Second response with a link

In [ ]:
# First response without link
link = json.loads(response.choices[0].message.tool_calls[0].function.arguments).get("link", None)
extract_live_info(link)

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information


{'timestamp': '2024-10-31T10:24:47',
 'location': 'Tuomiokirkko',
 'dimensions': {'width': 1280, 'height': 720},
 'buildings': {'number_of_buildings': 20,
  'building_height_range': '2-8 stories'},
 'vehicles': {'number_of_vehicles': 0, 'types': []},
 'waterbodies': {'visible': False, 'type': None, 'number_of_boats': 0},
 'street_lights': {'number_of_street_lights': 0},
 'people': {'approximate_number': 0},
 'lighting': {'time_of_day': 'morning', 'artificial_lighting': 'none'},
 'visibility': {'clear': True},
 'sky': {'visible': True, 'light_conditions': 'daylight'}}

In [ ]:
# Second response with a link
link = json.loads(response.choices[0].message.tool_calls[1].function.arguments).get("link", None)
extract_live_info(link)

[youtube] Extracting URL: https://www.youtube.com/watch?v=dFBRpHHwQeg
[youtube] dFBRpHHwQeg: Downloading webpage
[youtube] dFBRpHHwQeg: Downloading ios player API JSON
[youtube] dFBRpHHwQeg: Downloading mweb player API JSON
[youtube] dFBRpHHwQeg: Downloading m3u8 information
[youtube] dFBRpHHwQeg: Downloading m3u8 information


{'timestamp': '2024-10-31T09:29:40',
 'dimensions': {'width': 1280, 'height': 720},
 'buildings': {'number_of_buildings': 15,
  'building_height_range': '4-10 stories'},
 'waterbodies': {'visible': True, 'type': 'canal', 'number_of_boats': 10},
 'people': {'approximate_number': 50},
 'lighting': {'time_of_day': 'morning', 'artificial_lighting': 'none'},
 'visibility': {'clear': True},
 'sky': {'visible': True, 'light_conditions': 'daylight'}}

### Example for automated function call and GPT response

In [ ]:
index = 0

messages = [
    {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": "Hi, can you tell me the information in YouTube Live? I don't have any specific interest. Just give me some readily available one."}
]

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)

# Append the assistant's message
message = response.choices[0].message
messages.append(message)

# Check if a function call is in the message
if len(message.tool_calls) > 0:
    function_name = message.tool_calls[index].function.name
    arguments = json.loads(message.tool_calls[index].function.arguments)

    # Call the function dynamically
    if function_name in functions:
        result = functions[function_name](**arguments)
        # Append function result to messages
        messages.append({
            "role": "tool",
            "name": function_name,
            "content": json.dumps(result),
            "tool_call_id": message.tool_calls[index].id
        })

    index += 1

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information


In [ ]:
# Feed the function call to the chat and see the response
response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)
print(response.choices[0].message.content)

Here's some readily available information extracted from a YouTube Live:

- **Timestamp:** 2024-10-31T17:02:20
- **Dimensions:** 1280x720 pixels
- **Buildings:** 
  - Number of buildings: 5 
  - Height range: 2-6 stories
- **Vehicles:** Number of vehicles: 6
- **Waterbodies:** 
  - Visible: Yes
  - Type: Harbor
  - Number of boats: 1
- **Street Lights:** Number of street lights: 25
- **People:** Approximate number: 15
- **Lighting:** 
  - Time of day: Evening
  - Artificial lighting: Prominent
- **Visibility:** Clear
- **Sky:** Not visible

If you need more specific details or have a particular area of interest, feel free to ask!
